⚠️ **Note**: **Submission is currently non-functional, as the obfuscated holdout data has not been released yet. This will be resolved in time for the start of the competition.**

# 🍍 LibriBrain Competition: Submission
You've trained a model for one of our tracks and are now ready to submit your results? Congratulations! - let's walk through the process.

Broadly, you will need to do the following:
1. Register your team on Eval.ai (see the tutorial [here](https://evalai.readthedocs.io/en/latest/participate.html))
2. Run model predictions on our holdout data
3. Submit the .CSV file containing your results.

This tutorial will walk you through step (2), generating the .CSV file for you to submit.

In case of any questions or problems, please get in touch through [our Discord server](https://discord.gg/Fqr8gJnvSh).

⚠️ **Note**: We have only comprehensively validated the notebook to work on Colab and Unix. Your experience in other environments (e.g., Windows) may vary.

## Setting up dependencies
Run the code below *as is*. It will download all required dependencies, including our own [PNPL](https://pypi.org/project/pnpl/) package. On Windows, you might have to restart your Kernel after the installation has finished.

In [ ]:
# Install additional dependencies
%pip install -q mne_bids lightning torchmetrics scikit-learn plotly ipywidgets pnpl

# Set up base path for dataset and related files (base_path is assumed to be set in the cells below!)
base_path = "./libribrain"
try:
    import google.colab  # This module is only available in Colab.
    in_colab = True
    base_path = "/content"  # This is the folder displayed in the Colab sidebar
except ImportError:
    in_colab = False

# Loading our model
The code below sets up the model architecture of our two baselines models and loads pretrained weights (so we can skip training).
In your own submission, this is where you would swap in your own setup.
While we are demonstrating submission for both tracks (Speech Detection and Phoneme Classification), you will only need to set up a single model for one of the tracks here.

In [ ]:
import torch
import torch.nn as nn
import lightning as L
import torchmetrics
import requests
import os
import numpy as np
from lightning.pytorch.callbacks import Callback
from sklearn.metrics import roc_curve, auc, balanced_accuracy_score, jaccard_score

# Custom BCE loss with label smoothing
class BCEWithLogitsLossWithSmoothing(nn.Module):
    def __init__(self, smoothing=0.1, pos_weight=1.0):
        super().__init__()
        self.smoothing = smoothing
        self.bce_loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]))

    def forward(self, logits, target):
        target = target.float()  # ensure target is float
        target_smoothed = target * (1 - self.smoothing) + self.smoothing * 0.5
        return self.bce_loss(logits, target_smoothed)

#
# Speech Detection Model
#
class SpeechDetectionModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        # Fixed architecture parameters
        input_dim = 23         # e.g. len(SENSORS_SPEECH_MASK)
        model_dim = 100
        dropout_rate = 0.5
        lstm_layers = 2
        bi_directional = False
        batch_norm = False

        # Define convolutional block
        self.conv = nn.Conv1d(
            in_channels=input_dim,
            out_channels=model_dim,
            kernel_size=3,
            padding=1,
        )
        self.batch_norm = nn.BatchNorm1d(model_dim) if batch_norm else nn.Identity()
        self.conv_dropout = nn.Dropout(p=dropout_rate)

        # Define LSTM block (batch_first=True so that output shape is (batch, seq_len, features))
        self.lstm = nn.LSTM(
            input_size=model_dim,
            hidden_size=model_dim,
            num_layers=lstm_layers,
            dropout=dropout_rate,
            batch_first=True,
            bidirectional=bi_directional
        )
        self.lstm_dropout = nn.Dropout(p=dropout_rate)

        # Final classifier layer
        self.speech_classifier = nn.Linear(model_dim, 1)

        # Use our loss with smoothing
        self.loss_fn = BCEWithLogitsLossWithSmoothing(smoothing=0.1, pos_weight=1.0)

    def forward(self, x):
        # x shape: (batch, channels, seq_len)
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.conv_dropout(x)
        # Permute to (batch, seq_len, features) for LSTM
        x = x.permute(0, 2, 1)
        output, (h_n, c_n) = self.lstm(x)
        # Use the last hidden state from the final LSTM layer (shape: (batch, model_dim))
        hidden = h_n[-1]
        hidden = self.lstm_dropout(hidden)
        logits = self.speech_classifier(hidden)
        return logits

    def training_step(self, batch, batch_idx):
        x, y = batch  # assume y is (batch,) with binary labels
        logits = self(x)
        loss = self.loss_fn(logits, y.unsqueeze(1).float())
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y.unsqueeze(1).float())
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y.unsqueeze(1).float())
        self.log("test_loss", loss, prog_bar=True)
        return {"loss": loss, "logits": logits, "y": y}

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3, weight_decay=0.01)
        return optimizer


#
# Phoneme Classification Model
#
class PhonemeClassificationModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv1d(306, 128, 1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(16000, 39)
        )
        self.criterion = nn.CrossEntropyLoss()
        self.f1_macro = torchmetrics.F1Score(num_classes=39, average='macro', task="multiclass")

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        f1_macro = self.f1_macro(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_f1_macro', f1_macro)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        f1_macro = self.f1_macro(y_hat, y)
        self.log('val_loss', loss)
        self.log('val_f1_macro', f1_macro, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.0005)


# Set a fixed seed for reproducibility and initialize trainer
L.seed_everything(42)
trainer = L.Trainer(devices="auto")

CHECKPOINT_PATH_SPEECH = f"{base_path}/models/speech_model_minimal.ckpt"
CHECKPOINT_PATH_PHONEME = f"{base_path}/models/phoneme_model.ckpt"

# Download and load the speech model checkpoint if it does not already exist
if not os.path.exists(CHECKPOINT_PATH_SPEECH):
    speech_model_url = "https://neural-processing-lab.github.io/2025-libribrain-competition/speech_model_minimal.ckpt"
    response = requests.get(speech_model_url)
    os.makedirs(os.path.dirname(CHECKPOINT_PATH_SPEECH), exist_ok=True)
    with open(CHECKPOINT_PATH_SPEECH, "wb") as f:
        f.write(response.content)
    print("Download of speech model checkpoint complete.")
else:
    print("Speech model checkpoint already exists. Skipping download.")

# Load the unified SpeechDetectionModel from checkpoint
speech_model = SpeechDetectionModel.load_from_checkpoint(
    checkpoint_path=CHECKPOINT_PATH_SPEECH
)

# Download and load the phoneme model checkpoint if it does not already exist
if not os.path.exists(CHECKPOINT_PATH_PHONEME):
    phoneme_model_url = "https://neural-processing-lab.github.io/2025-libribrain-competition/phoneme_model.ckpt"
    response = requests.get(phoneme_model_url)
    os.makedirs(os.path.dirname(CHECKPOINT_PATH_PHONEME), exist_ok=True)
    with open(CHECKPOINT_PATH_PHONEME, "wb") as f:
        f.write(response.content)
    print("Download of phoneme model checkpoint complete.")
else:
    print("Phoneme model checkpoint already exists. Skipping download.")

# Load the PhonemeClassificationModel from checkpoint
phoneme_model = PhonemeClassificationModel.load_from_checkpoint(
    checkpoint_path=CHECKPOINT_PATH_PHONEME
)


# Generating submission file
Now that we have our models loaded, let's generate the submission CSV.

### Speech detection submission

In [ ]:
import lightning as L
import requests
import os
from torch.utils.data import DataLoader
from pnpl.datasets import LibriBrainCompetitionHoldout


holdout_dataset = LibriBrainCompetitionHoldout(base_path, task="speech")
holdout_loader = DataLoader(holdout_dataset, batch_size=1, num_workers=0,shuffle=False)

# Generate predictions for the holdout dataset
predictions = trainer.predict(speech_model, dataloaders=holdout_loader)
# Submit predictions to CSV
holdout_dataset.generate_submission_in_csv(predictions=predictions, output_path="speech_predictions.csv")

### Phoneme classification submission

In [ ]:
import lightning as L
import requests
import os
from torch.utils.data import DataLoader
from pnpl.datasets import LibriBrainCompetitionHoldout


holdout_dataset = LibriBrainCompetitionHoldout(base_path, task="phoneme")
holdout_loader = DataLoader(holdout_dataset, batch_size=1, num_workers=0,shuffle=False)

# Generate predictions for the holdout dataset
predictions = trainer.predict(phoneme_model, dataloaders=holdout_loader)
# Submit predictions to CSV
holdout_dataset.generate_submission_in_csv(predictions=predictions, output_path="phoneme_predictions.csv")

## That's it! 🥳
You've successfully generated a submission .CSV file. Two steps remain:
1. Download the file by selecting `Files -> speech_predictions.csv -> Download` or `Files -> phoneme_predications.csv -> Download` on the toolbar on the left
2. Upload the file on EvalAI to secure your spot on the leaderboard.

If you have any open questions, get in touch on [our Discord server](https://discord.gg/Fqr8gJnvSh) or find more information on [our website](https://neural-processing-lab.github.io/2025-libribrain-competition). Good luck!